In [1]:
# import dependencies
import cv2
import mediapipe as mp
import numpy as np
import math

In [2]:
# initialize the mediapipe poses and drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
def calculate_angle(a, b, c):
    # find vector between AB (vector u) and AC (vector v)
    u = ((a.x - b.x), (a.y - b.y))
    v = ((a.x - c.x), (a.y - c.y))

    # dot_product
    dot_uv = np.dot(u, v)

    # magnitude of u
    mag_u = math.sqrt(pow(u[0], 2) + pow(u[1], 2))
    mag_v = math.sqrt(pow(v[0], 2) + pow(v[1], 2))

    # magnitude of uv
    mag_uv = mag_u * mag_v
   
    theta_rad = math.acos(dot_uv / mag_uv)
  
    # radians to degrees
    theta_deg = math.degrees(theta_rad)
    return round(theta_deg, 2)

In [5]:
# setup mediapipe
cap = cv2.VideoCapture(0)
r_elbow_angle = []
l_elbow_angle = []

# check if camera can open
if not cap.isOpened():
    print("Could not open camera")
    exit()
    
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # invert the video
        frame = cv2.flip(frame, 1)
        
        # recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False       # makes image read-only for detection
        
        # make the detection
        results = pose.process(image)
        
        # recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # get landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # get coordinates right shoulder
            r_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            r_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            r_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            # get coordinates left shoulder
            l_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            l_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            l_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            
            # calculate angle
            r_angle = calculate_angle(r_elbow, r_wrist, r_shoulder)
            l_angle = calculate_angle(l_elbow, l_wrist, l_shoulder)
            
            r_elbow_angle.append(r_angle)
            l_elbow_angle.append(l_angle)
            
            # visualize
            cv2.putText(image, str(r_angle), tuple(np.multiply((r_elbow.x, r_elbow.y), [640, 480]).astype(int)), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(l_angle), tuple(np.multiply((l_elbow.x, l_elbow.y), [640, 480]).astype(int)), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2, cv2.LINE_AA)
        except AttributeError:
            pass
        
        # render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(0, 0, 255)), mp_drawing.DrawingSpec(color=(0, 255, 0)))
        
        cv2.imshow('Backend Limb Detection', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
                
cv2.destroyAllWindows()
cap.release()

In [ ]:
len(landmarks)

In [ ]:
for lndmk in mp_pose.PoseLandmark:
    print(lndmk)

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
# landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x

In [ ]:
mp_pose.PoseLandmark.LEFT_ELBOW.value

In [ ]:
# # for testing
# def calculate_angle(a, b, c):
#     # find vector between AB (vector u) and AC (vector v)
#     u = ((b[0] - a[0]), (b[1] - a[1]))
#     v = ((c[0] - a[0]), (c[1] - a[1]))
#     print("vectors")
#     print(u)
#     print(v)
#     
#     # dot_product
#     dot_uv = np.dot(u, v)
#     print("dot uv")
#     print(dot_uv)
#     
#     # magnitude of u
#     mag_u = math.sqrt(pow(u[0], 2) + pow(u[1], 2))
#     mag_v = math.sqrt(pow(v[0], 2) + pow(v[1], 2))
#     print("mag u and v")
#     print(mag_u, mag_v)
#     
#     # magnitude of uv
#     mag_uv = mag_u * mag_v
#     print("mag uv")
#     print(mag_uv)
#     
#     theta_rad = math.acos(dot_uv / mag_uv)
#     print("theta")
#     print(theta_rad)
#     
#     # radians to degrees
#     theta_deg = math.degrees(theta_rad)
#     print("theta_deg")
#     print(theta_deg)
# 
# calculate_angle((-1,1), (2,-1), (2,2))